In [1]:
# Python built-ins
import sys

# FlexRec
import flexrec
from flexrec.features import QueryFeature
from flexrec.models import QueryModel, QueryReduction, RetrievalModel, RankingModel
from flexrec.trainers import MultitaskTrainer

# Vectorization
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Check Python & Tensorflow version
# Developed using Python 3.6.10 & TensorFlow 2.6.0
print('Python {}'.format(sys.version))
print('TensorFlow {}'.format(tf.__version__))

# Check processing units
# Troubleshoot 1: if no GPU is detected: https://www.tensorflow.org/install/gpu)
# Troubleshoot 2: if libcusolver.so.11 is not found: use ln -s to create alias from libcusolver.so.10
print('Physical devices: {}'.format(tf.config.list_physical_devices()))

Python 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
TensorFlow 2.6.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
# Load Movielens dataset (online)

# Ratings data
ratings = tfds.load('movielens/100k-ratings', split="train") # For Movielens, only "train" split is available

# Features of all the available movies
movies = tfds.load('movielens/100k-movies', split="train")

# Check one record from each dataset
for x in ratings.take(1).as_numpy_iterator(): print(x)
for x in movies.take(1).as_numpy_iterator(): print(x)

{'bucketized_user_age': 45.0, 'movie_genres': array([7]), 'movie_id': b'357', 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'raw_user_age': 46.0, 'timestamp': 879024327, 'user_gender': True, 'user_id': b'138', 'user_occupation_label': 4, 'user_occupation_text': b'doctor', 'user_rating': 4.0, 'user_zip_code': b'53211'}
{'movie_genres': array([4]), 'movie_id': b'1681', 'movie_title': b'You So Crazy (1994)'}


In [3]:
# Data preprocessing

# Filter unnecessary features & convert types
ratings = ratings.map(lambda x: {
            "bucketized_user_age": int(x["bucketized_user_age"]),
            "movie_id": x["movie_id"],
            "movie_title": x["movie_title"],
            "timestamp": float(x["timestamp"]),
            "user_gender": int(x["user_gender"]),
            "user_id": x["user_id"],
            "user_occupation_label": x["user_occupation_label"],
            "label": x["user_rating"],
            "user_zip_code": x["user_zip_code"],
          })
movies = movies.map(lambda x: {
           "movie_id": x["movie_id"],
           "movie_title": x["movie_title"],
         })

# Shuffle
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# Split to train/test sets (in real applications, this must be done by dividing on time T)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Cache dataset for efficiency
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [4]:
# Set up query feature configurations for building embedding tables
user_features = [
  QueryFeature('user_id', ratings.map(lambda x: x['user_id']), QueryFeature.STR_TYPE),
  QueryFeature('user_zip_code', ratings.map(lambda x: x['user_zip_code']), QueryFeature.STR_TYPE),
  QueryFeature('bucketized_user_age', ratings.map(lambda x: x['bucketized_user_age']), QueryFeature.INT_TYPE),
  QueryFeature('user_gender', ratings.map(lambda x: x['user_gender']), QueryFeature.INT_TYPE),
  QueryFeature('user_occupation_label', ratings.map(lambda x: x['user_occupation_label']), QueryFeature.INT_TYPE),
  QueryFeature('timestamp', ratings.map(lambda x: x['timestamp']), QueryFeature.CDI_TYPE),
  QueryFeature('timestamp', ratings.map(lambda x: x['timestamp']), QueryFeature.CNO_TYPE)
]
movie_features = [
  QueryFeature('movie_id', movies.map(lambda x: x['movie_id']), QueryFeature.STR_TYPE),
  QueryFeature('movie_title', movies.map(lambda x: x['movie_title']), QueryFeature.TXT_TYPE)
]

In [5]:
# Build query towers
embedding_dim = 32

user_model = QueryModel(features=[user_features[0], user_features[3]], embedding_dim=embedding_dim)
movie_model = QueryModel(features=movie_features, embedding_dim=embedding_dim)

for row in ratings.batch(1).take(1):
  print(user_model(row).shape, movie_model(row).shape)

(1, 2, 32) (1, 2, 32)


In [6]:
# Build retrieval model and train
layer_sizes = None
normalization = 0.9
regularization = 1e-7
gravitation = 1e-8

learning_rate = 0.2
num_epochs = 50

retrieval_model = RetrievalModel(user_model, movie_model, movies, 
                                 layer_sizes=layer_sizes,
                                 normalization=normalization,
                                 regularization=regularization, 
                                 gravitation=gravitation)
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))
retrieval_model.fit(cached_train, epochs=num_epochs, validation_data=cached_test, validation_freq=10)

Epoch 1/50
10/10 [==============================] - 6s 85ms/step - cosine_loss: 70305.1357 - gravitation_loss: 5.7435e-04 - regularization_loss: 2.1786e-06 - total_loss: 70305.1357
Epoch 2/50
10/10 [==============================] - 0s 36ms/step - cosine_loss: 68747.4169 - gravitation_loss: 3.7073e-04 - regularization_loss: 4.7545e-06 - total_loss: 68747.4169
Epoch 3/50
10/10 [==============================] - 0s 36ms/step - cosine_loss: 68015.7294 - gravitation_loss: 4.9406e-04 - regularization_loss: 6.6756e-06 - total_loss: 68015.7294
Epoch 4/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67831.7642 - gravitation_loss: 5.9099e-04 - regularization_loss: 7.9978e-06 - total_loss: 67831.7642
Epoch 5/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67737.5227 - gravitation_loss: 6.7740e-04 - regularization_loss: 9.1769e-06 - total_loss: 67737.5227
Epoch 6/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67676.1889 - gr

Epoch 41/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67263.2891 - gravitation_loss: 0.0027 - regularization_loss: 3.7799e-05 - total_loss: 67263.2898
Epoch 42/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67258.8793 - gravitation_loss: 0.0028 - regularization_loss: 3.8478e-05 - total_loss: 67258.8800
Epoch 43/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67254.5732 - gravitation_loss: 0.0028 - regularization_loss: 3.9154e-05 - total_loss: 67254.5739
Epoch 44/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67250.3693 - gravitation_loss: 0.0029 - regularization_loss: 3.9827e-05 - total_loss: 67250.3700
Epoch 45/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67246.2607 - gravitation_loss: 0.0029 - regularization_loss: 4.0497e-05 - total_loss: 67246.2614
Epoch 46/50
10/10 [==============================] - 0s 35ms/step - cosine_loss: 67242.2450 - gravitation_loss

In [7]:
# Build ranking model and train
layer_sizes = [192, 192, 192]
use_interaction = True
projection_dim = None
regularization = 1e-1
label_name = 'label'

learning_rate = 0.05
num_epochs = 50

for row in ratings.batch(1).take(1):
  print(user_model(row).shape, movie_model(row).shape)

ranking_model = RankingModel(user_model, movie_model, 
                             layer_sizes=layer_sizes,
                             use_interaction=use_interaction,
                             projection_dim=projection_dim,
                             regularization=regularization,
                             label_name=label_name)
ranking_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
ranking_model.fit(cached_train, epochs=num_epochs, validation_data=cached_test, validation_freq=10)

(1, 2, 32) (1, 2, 32)
Epoch 1/50
10/10 [==============================] - 2s 19ms/step - loss: 7749.9256 - regularization_loss: 36.1485 - total_loss: 7786.0742
Epoch 2/50
10/10 [==============================] - 0s 19ms/step - loss: 15.5353 - regularization_loss: 32.5193 - total_loss: 48.0547
Epoch 3/50
10/10 [==============================] - 0s 19ms/step - loss: 14.8467 - regularization_loss: 32.0098 - total_loss: 46.8565
Epoch 4/50
10/10 [==============================] - 0s 19ms/step - loss: 14.6971 - regularization_loss: 31.3888 - total_loss: 46.0859
Epoch 5/50
10/10 [==============================] - 0s 19ms/step - loss: 14.4308 - regularization_loss: 30.6716 - total_loss: 45.1024
Epoch 6/50
10/10 [==============================] - 0s 19ms/step - loss: 14.1031 - regularization_loss: 29.9725 - total_loss: 44.0756
Epoch 7/50
10/10 [==============================] - 0s 20ms/step - loss: 28.1299 - regularization_loss: 29.3508 - total_loss: 57.4807
Epoch 8/50
10/10 [==================

In [8]:
learning_rate = 0.05
num_epochs = 50
retrieval_weight = 1.0
ranking_weight = 0.5

multitask_trainer = MultitaskTrainer([retrieval_model], [ranking_model], [retrieval_weight], [ranking_weight])
multitask_trainer.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
multitask_trainer.fit(cached_train, epochs=num_epochs, validation_data=cached_test, validation_freq=10)

Epoch 1/50
10/10 [==============================] - 2s 37ms/step - retrieval_loss: 68161.5675 - ranking_loss: 24.2104 - total_loss: 68185.7777
Epoch 2/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67465.1087 - ranking_loss: 29.1655 - total_loss: 67494.2734
Epoch 3/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67354.9964 - ranking_loss: 31.5208 - total_loss: 67386.5178
Epoch 4/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67307.9325 - ranking_loss: 36.1045 - total_loss: 67344.0362
Epoch 5/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67275.1776 - ranking_loss: 39.8990 - total_loss: 67315.0767
Epoch 6/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67248.4851 - ranking_loss: 43.3829 - total_loss: 67291.8679
Epoch 7/50
10/10 [==============================] - 0s 36ms/step - retrieval_loss: 67225.8253 - ranking_loss: 46.7489 - total_loss: 67272.5753